## Setup

Import required libraries:

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

print("✅ All imports successful!")

---

## 🏗️ Part 1: Circuit Creation Methods (EXAM CRITICAL!)

### Method 1: Simple Creation (Most Common)

**When to use**: 90% of the time, this is what you need!

In [ ]:
# Just qubits (no classical bits needed yet)
qc1 = QuantumCircuit(2)
print("Circuit with 2 qubits:")
print(qc1.draw())

print("\n" + "="*50 + "\n")

# Qubits + classical bits (for measurement)
qc2 = QuantumCircuit(2, 2)
qc2.h(0)
qc2.cx(0, 1)
qc2.measure([0, 1], [0, 1])

print("Circuit with 2 qubits and 2 classical bits:")
print(qc2.draw())

print("\n✅ Syntax: QuantumCircuit(n_qubits, n_classical_bits)")

### Method 2: Using Named Registers

**When to use**: Complex circuits where you want better organization and readability

In [ ]:
# Create named registers
qr = QuantumRegister(3, 'q')   # 3 qubits named 'q'
cr = ClassicalRegister(3, 'c')  # 3 classical bits named 'c'

qc = QuantumCircuit(qr, cr)
qc.h(qr[0])
qc.cx(qr[0], qr[1])
qc.cx(qr[1], qr[2])
qc.measure(qr, cr)

print("Circuit with named registers:")
print(qc.draw())

print("\n✅ Benefits:")
print("   - Better organization")
print("   - Named registers improve readability")
print("   - Can use register indexing: qr[0], qr[1], etc.")


### Method 3: Multiple Registers (Advanced)

**When to use**: Error correction, complex algorithms with different logical units

In [ ]:
# Separate logical units
data_qubits = QuantumRegister(4, 'data')
ancilla_qubits = QuantumRegister(2, 'ancilla')
measurements = ClassicalRegister(4, 'meas')
syndrome = ClassicalRegister(2, 'syndrome')

qc = QuantumCircuit(data_qubits, ancilla_qubits, measurements, syndrome)

# Operations on specific registers
qc.h(data_qubits)  # Hadamard on all data qubits
qc.x(ancilla_qubits[0])  # X on first ancilla

# Measure to specific registers
qc.measure(data_qubits, measurements)
qc.measure(ancilla_qubits, syndrome)

print("Circuit with multiple registers:")
print(qc.draw())

print("\n✅ Use cases:")
print("   - Error correction codes")
print("   - Quantum algorithms with ancilla qubits")
print("   - Logical grouping for complex circuits")


---

## 📏 Part 2: Circuit Properties (EXAM CRITICAL!)

### The Big Three: Depth, Size, Width

**These are tested in EVERY exam!**

In [ ]:
# Create a GHZ state circuit
qc = QuantumCircuit(3, 3)
qc.h(0)              # Layer 1
qc.cx(0, 1)          # Layer 2
qc.cx(1, 2)          # Layer 3
qc.measure([0,1,2], [0,1,2])  # Layer 4

print("GHZ State Circuit:")
print(qc.draw())

print("\n" + "="*50)
print("\n📊 CIRCUIT PROPERTIES (MEMORIZE THESE!)")
print("="*50)

print(f"\n1. Depth:      {qc.depth()}")
print("   → Critical path length (longest sequential chain)")
print("   → Layers: H → CX(0,1) → CX(1,2) → Measure = 4 layers")

print(f"\n2. Size:       {qc.size()}")
print("   → Total gate count")
print("   → Count: 1 H + 2 CX + 3 Measure = 6 operations")

print(f"\n3. Width:      {qc.width()}")
print("   → Total wires (qubits + classical bits)")
print("   → Count: 3 qubits + 3 classical = 6 wires")

print(f"\n4. Num Qubits: {qc.num_qubits}")
print("   → Number of quantum wires")
print("   ⚠️  PROPERTY (no parentheses!)")

print(f"\n5. Num Clbits: {qc.num_clbits}")
print("   → Number of classical wires")
print("   ⚠️  PROPERTY (no parentheses!)")

print(f"\n6. Count Ops:  {qc.count_ops()}")
print("   → Dictionary of gate types and counts")


### ⚠️ EXAM TRAP #1: Property vs Method

**This is tested CONSTANTLY on the exam!**

```python
# ❌ WRONG - These are properties (no parentheses!)
qc.num_qubits()   # ERROR!
qc.num_clbits()   # ERROR!

# ✅ CORRECT - Properties
qc.num_qubits     # No ()
qc.num_clbits     # No ()

# ✅ CORRECT - Methods  
qc.depth()        # With ()
qc.size()         # With ()
qc.width()        # With ()
qc.count_ops()    # With ()
```

**Memory aid**: "Numbers are properties, actions are methods!"

In [ ]:
# Test yourself - which will work?
print("Testing property vs method:")

try:
    print(f"✅ qc.num_qubits = {qc.num_qubits}")
except Exception as e:
    print(f"❌ Error: {e}")

try:
    print(f"✅ qc.depth() = {qc.depth()}")
except Exception as e:
    print(f"❌ Error: {e}")

# This will fail:
try:
    result = qc.num_qubits()
    print(f"Result: {result}")
except Exception as e:
    print(f"❌ qc.num_qubits() fails: {type(e).__name__}")
    print("   → Use qc.num_qubits (no parentheses!)")

---

## 🎯 Part 3: Understanding Depth vs Size (EXAM CRITICAL!)

### Parallel Operations Don't Add Depth!

In [ ]:
# Example 1: Parallel operations
qc_parallel = QuantumCircuit(3)
qc_parallel.h(0)  # Layer 1
qc_parallel.h(1)  # Layer 1 (parallel!)
qc_parallel.h(2)  # Layer 1 (parallel!)

print("Parallel Hadamards:")
print(qc_parallel.draw())
print(f"\nDepth: {qc_parallel.depth()}  ← Only 1! (all gates in same layer)")
print(f"Size:  {qc_parallel.size()}   ← 3 gates total")
print(f"Width: {qc_parallel.width()}  ← 3 qubits")

In [ ]:
# Example 2: Sequential operations
qc_sequential = QuantumCircuit(1)
qc_sequential.h(0)  # Layer 1
qc_sequential.x(0)  # Layer 2
qc_sequential.z(0)  # Layer 3

print("Sequential Gates on Same Qubit:")
print(qc_sequential.draw())
print(f"\nDepth: {qc_sequential.depth()}  ← 3! (each gate in different layer)")
print(f"Size:  {qc_sequential.size()}   ← 3 gates total")
print(f"Width: {qc_sequential.width()}  ← 1 qubit")

In [ ]:
# Example 3: Mixed (common exam question!)
qc_mixed = QuantumCircuit(3)
qc_mixed.h(0)       # Layer 1
qc_mixed.h(1)       # Layer 1 (parallel with above)
qc_mixed.cx(0, 2)   # Layer 2 (depends on q0)
qc_mixed.x(1)       # Layer 2 (parallel with CX, different qubits)

print("Mixed Parallel and Sequential:")
print(qc_mixed.draw())
print(f"\nDepth: {qc_mixed.depth()}")
print(f"Size:  {qc_mixed.size()}")
print("\nAnalysis:")
print("  Layer 1: H(0) and H(1) happen together")
print("  Layer 2: CX(0,2) and X(1) happen together")
print("  → Depth = 2 layers")
print("  → Size = 4 gates total")

### Calculating Depth - Step by Step

**Algorithm for calculating depth manually**:

1. Go through gates in order
2. Track which qubits are "busy" in each layer
3. If gate uses a qubit that's busy → new layer
4. If all qubits are free → same layer
5. Count total layers

In [ ]:
# Practice example
qc_practice = QuantumCircuit(4)
qc_practice.h(0)        # Layer 1
qc_practice.h(1)        # Layer 1 (parallel)
qc_practice.h(2)        # Layer 1 (parallel)
qc_practice.cx(0, 1)    # Layer 2 (0 and 1 busy)
qc_practice.cx(2, 3)    # Layer 2 (parallel! different qubits)
qc_practice.x(0)        # Layer 3 (0 was busy in layer 2)

print("Practice Circuit:")
print(qc_practice.draw())
print(f"\nDepth: {qc_practice.depth()}")
print(f"Size:  {qc_practice.size()}")

print("\nLayer breakdown:")
print("  Layer 1: H(0), H(1), H(2) - all parallel")
print("  Layer 2: CX(0,1), CX(2,3) - parallel, different qubit pairs")
print("  Layer 3: X(0) - waits for CX(0,1) to finish")

---

## 🧪 Part 4: count_ops() - Operation Breakdown

In [ ]:
# Create a varied circuit
qc = QuantumCircuit(3, 3)
qc.h([0, 1, 2])
qc.cx(0, 1)
qc.cx(1, 2)
qc.x(0)
qc.z(1)
qc.measure([0, 1, 2], [0, 1, 2])

print("Complex Circuit:")
print(qc.draw())

ops = qc.count_ops()
print(f"\nOperation count: {ops}")
print("\nBreakdown:")
for gate, count in ops.items():
    print(f"  {gate}: {count}")

print(f"\nTotal operations (size): {qc.size()}")
print(f"Verify: {sum(ops.values())} = {qc.size()} ✅")

---

## 📝 Practice Questions

### Question 1: Depth Calculation

**What is the depth of this circuit?**

```python
qc = QuantumCircuit(3)
qc.h(0)
qc.h(1)
qc.cx(0, 2)
qc.x(1)
```

A) 1  
B) 2  
C) 3  
D) 4

<details>
<summary>Click to reveal answer</summary>

**Answer: B) 2**

**Explanation**:

Layer breakdown:
- **Layer 1**: H(0) and H(1) - parallel (different qubits)
- **Layer 2**: CX(0,2) and X(1) - parallel (different qubits)

CX(0,2) doesn't conflict with X(1) because they operate on different qubits!

**Depth = 2 layers**
</details>

---

### Question 2: Property vs Method

**Which code is CORRECT?**

A) `n = qc.num_qubits()`  
B) `d = qc.depth`  
C) `w = qc.width()`  
D) `s = qc.size`

<details>
<summary>Click to reveal answer</summary>

**Answer: C) `w = qc.width()`**

**Explanation**:

- A) ❌ `num_qubits` is a PROPERTY (no parentheses)
- B) ❌ `depth` is a METHOD (needs parentheses)
- C) ✅ `width()` is a METHOD (correct!)
- D) ❌ `size` is a METHOD (needs parentheses)

**Correct versions**:
```python
n = qc.num_qubits     # Property
d = qc.depth()        # Method
w = qc.width()        # Method
s = qc.size()         # Method
```

**Memory aid**: "Numbers are properties (num_qubits, num_clbits), calculations are methods!"
</details>

---

### Question 3: Width Calculation

**What is the width of `QuantumCircuit(5, 3)`?**

A) 3  
B) 5  
C) 8  
D) 15

<details>
<summary>Click to reveal answer</summary>

**Answer: C) 8**

**Explanation**:

Width = Total wires = Qubits + Classical bits

Width = 5 qubits + 3 classical bits = **8 wires total**

```python
qc = QuantumCircuit(5, 3)
print(qc.width())  # 8
print(qc.num_qubits)  # 5
print(qc.num_clbits)  # 3
```
</details>

---

## ✅ Key Takeaways - Master These for the Exam!

### Core Concepts

1. **Circuit Creation**
   - Simple: `QuantumCircuit(n_qubits, n_clbits)`
   - Registers: `QuantumCircuit(QuantumRegister, ClassicalRegister)`
   - Multiple registers for complex circuits

2. **Properties (CRITICAL!)**
   - `depth()` - Critical path length (longest sequential chain)
   - `size()` - Total gate count
   - `width()` - Total wires (qubits + classical)
   - `num_qubits` - Property (no parentheses!)
   - `num_clbits` - Property (no parentheses!)
   - `count_ops()` - Dictionary of operation counts

3. **Depth vs Size**
   - Parallel operations → same depth, increase size
   - Sequential operations → increase both
   - Calculate depth by counting layers, not gates!

### Critical Exam Facts

- ✅ `num_qubits` and `num_clbits` are PROPERTIES (no parentheses)
- ✅ `depth()`, `size()`, `width()`, `count_ops()` are METHODS (with parentheses)
- ✅ Depth = longest path, NOT total gates
- ✅ Width = qubits + classical bits
- ✅ Parallel gates don't increase depth!
- ✅ `QuantumCircuit(n, m)` creates n qubits and m classical bits

### Mnemonics to Remember

- 🧠 **"Width = Wires, Depth = Delays, Size = Sum"**
- 🧠 **"Numbers are Properties, Calculations are Methods"**
- 🧠 **"Parallel = Same Layer, Sequential = New Layer"**

### Common Exam Patterns

**Q**: What's the depth of this circuit with parallel gates?  
**A**: Count layers, not gates!

**Q**: Is `num_qubits` a property or method?  
**A**: Property (no parentheses)

**Q**: What's the width of QuantumCircuit(3, 2)?  
**A**: 3 + 2 = 5

**Q**: How to create circuit with named registers?  
**A**: `qc = QuantumCircuit(QuantumRegister(n, 'name'), ClassicalRegister(m, 'name'))`

### What's Next?

**Circuit Composition** - Learn compose(), append(), tensor() to build complex circuits from simple pieces!